julie.tsitron@parks.nyc.gov 2/20/2020

# Imports and Connections to DBs

In [1]:
pwd

'C:\\Projects\\Structures_test\\develop'

In [2]:
import pyproj

In [3]:
import geopandas

In [4]:
geopandas.__version__

'0.8.1'

In [5]:
pyproj.__version__

'2.6.1.post1'

In [6]:
# pyproj.

In [7]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import json
import urllib
import sqlalchemy
import os
import shapely
from datetime import datetime

from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor, SpatialDataFrame, FeatureLayer, FeatureSet

sys.path.append('../')
from IPM_Shared_Code_public.Python.geo_functions import read_geosql
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.email_functions import get_contacts, read_template, send_email
from geosupport import *

In [8]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
parksgis = config['srv']['parksgis']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']
portal = config['url']['portal']
structures_url = config['url']['structures']
geosupport_url = config['url']['geosupport']

In [9]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
                                 ';Database=' + structuresdb +
                                 ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection = engine.connect()

In [10]:
con_data_parks = pyodbc.connect('Driver={' + driver + '};Server=' + data_parks_server +
                     ';Database=IPMDB;Trusted_Connection=Yes;')
con = pyodbc.connect('Driver={' + driver + '};Server=' + server +
                           ';Database=;Trusted_Connection=Yes;')

## Deal with Dates

### m/d/Y H:M:S format:

In [11]:
# structures_dev['COMMISSIONDATE'] = pd.to_datetime(
#     structures_dev['COMMISSIONDATE'],
#     errors='coerce').dt.strftime('%m/%d/%Y %H:%M:%S')

In [12]:
# structures_dev['COMMISSIONDATE']

In [13]:
# structures_dev['FEATURESTATUSCHANGEDATE'] = pd.to_datetime(
#     structures_dev['FEATURESTATUSCHANGEDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [14]:
# structures_dev['RETIREDDATE'] = pd.to_datetime(
#     structures_dev['RETIREDDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [15]:
# structures_dev.columns.values

# Delta Table from structuresdb

In [16]:
# SPATIAL DATASET:
sql_str_deltas = 'select * FROM [structuresdb].[dbo].[tbl_delta_structures]'
struct_deltas = read_geosql(sql_str_deltas,
                            con,
                            geom_raw='shape',
                            geom_col='geometry')

In [18]:
## Still need this ??

struct_deltas.rename(columns={
    'objectid': 'OBJECTID',
    'parks_id': 'SYSTEM',
    'bin': 'BIN',
    'bbl': 'BBL',
    'doitt_id': 'DOITT_ID',
    'ground_elevation': 'Ground_Elevation',
    'height_roof': 'Height_Roof',
    'alteration_year': 'Alteration_Year',
    'construction_year': 'Construction_Year',
    'demolition_year': 'Demolition_Year'
},
                     inplace=True)

In [19]:
struct_deltas.head()

,fid,OBJECTID,SYSTEM,BIN,BBL,DOITT_ID,Ground_Elevation,Height_Roof,Construction_Year,Alteration_Year,Demolition_Year,api_call,doitt_source,date_stamp,geometry
0,1,6771,Q471-BLG0001,4542932,4000060001,1268580,8,21.687771,2013.0,NaN,None,U,current,2020-12-13 09:03:18.970,"POLYGON ((995222.165 209600.351, 995225.092 20..."
1,2,4812,Q343-BLG0512,4610410,4045380001,1268613,22,15.515073,1951.0,NaN,None,U,current,2020-12-13 09:03:18.970,"POLYGON ((1037340.009 227844.415, 1037318.921 ..."
2,3,12064,Q458-BLG0438,4610438,4059170001,1268811,34,11.029054,2011.0,NaN,None,U,current,2020-12-13 09:03:18.970,"POLYGON ((1046742.424 227957.080, 1046733.859 ..."
3,4,12028,M028-BLG9799,1089799,1021780600,1268900,-2,21.029465,NaN,NaN,None,U,current,2020-12-13 09:03:18.970,"POLYGON ((1002870.036 255245.035, 1002882.032 ..."
4,5,12065,X010-ZN03-BLG8179,2118179,2029420001,1269094,99,16.116720,2015.0,NaN,None,U,current,2020-12-13 09:03:18.970,"POLYGON ((1013172.907 244998.838, 1013173.492 ..."


In [28]:
struct_deltas[struct_deltas['SYSTEM']=='Q015-ZN06-BLG0418']

,fid,OBJECTID,SYSTEM,BIN,BBL,DOITT_ID,Ground_Elevation,Height_Roof,Construction_Year,Alteration_Year,Demolition_Year,api_call,doitt_source,date_stamp,geometry
618,626,14955,Q015-ZN06-BLG0418,4439149,4038660070,29096,151,19.0,1931.0,NaN,None,U,current,2020-12-13 09:03:18.970,"POLYGON ((1023470.614 194759.839, 1023455.125 ..."


In [21]:
multipolygons = []

for i, row in struct_deltas.iterrows(): 
    if type(row['geometry'])==shapely.geometry.multipolygon.MultiPolygon:
        multipolygons.append(row['SYSTEM'])

if len(multipolygons)!=0:
    ids = '\n'.join(multipolygons)
    mssg = 'SYSTEM ID(s) of building(s) that are MultiPolygons:\n'+ids
    send_email('mycontacts.txt','multipoly_mssg.txt', subject = 'multipolygons', e=mssg)
    
struct_deltas = struct_deltas[~struct_deltas['SYSTEM'].isin(multipolygons)]

In [22]:
multipolygons

[]

## Write Delta Table to geojson file

In [23]:
today = datetime.now().strftime('%Y%m%d')
snapshot = r'C:\\Projects\\Structures\\delta_snapshots/' + today

In [24]:
if not struct_deltas.empty:
    if os.path.exists(snapshot):
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')
    else:
        os.makedirs(snapshot)
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')

## Read geojson file to geojson object 

In [25]:
if not struct_deltas.empty:
    with open(snapshot+'/deltas.geojson') as data:
        geojson_deltas = json.load(data)

## Create arcgis featureSet from geojson object

In [43]:
type(struct_deltas)

geopandas.geodataframe.GeoDataFrame

In [44]:
struct_deltas.crs

<Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: USA - New York - SPCS - Long Island
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [57]:
# fromDF_deltas = FeatureSet.from_dataframe(pd.DataFrame(struct_deltas))

In [135]:
if not struct_deltas.empty:
    fromJSON_deltas = FeatureSet.from_geojson(geojson_deltas)

C:\ProgramData\Anaconda3\envs\structures_vpipm\lib\site-packages\arcgis\features\feature.py:799: UserWarning: crs has been deprecated and will be ignored. Please see: https://tools.ietf.org/html/rfc7946#section-4 for more information.
  warn("crs has been deprecated and will be ignored. Please"+ \


In [150]:
#Check the WKT Projection for the feature set
if len(fromJSON_deltas) > 0:
    print(fromJSON_deltas.spatial_reference)

{'wkid': 4326}


In [143]:
#Set the WKT projection for each feature in the feature set
for f in range(0, len(fromJSON_deltas)):
    fromJSON_deltas.features[f].geometry.update({'spatialReference': {'wkid': 2263}})

In [151]:
#Set the WKT projection for the whole feature set
fromJSON_deltas.spatial_reference = {'wkid': 2263}

In [152]:
#Check the WKT Projection for the feature set
if len(fromJSON_deltas) > 0:
    print(fromJSON_deltas.spatial_reference)

{'wkid': 2263}


# Connect to published datasets via GIS object

In [139]:
gis = GIS(url=portal, verify_cert = False)

# Make Edits

## Structures

### run delta archive stored proc

In [140]:
crsr = con.cursor()
sqal_stmt = '''
exec [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] 
'''
crsr.execute("{CALL [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] }")
crsr.commit()
crsr.close()

### Connect to structures feature layer directly

In [129]:
strct_lyr_url = structures_url

In [130]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

In [131]:
structures_features

<FeatureSet> 2709 features

In [132]:
len(struct_deltas)

802

### EDIT

In [141]:
if not struct_deltas.empty:
    num_updates = len(fromJSON_deltas.features)

In [149]:
if not struct_deltas.empty:
    try:
        update_result = lyr_structures.edit_features(
            updates=fromJSON_deltas.features)
    except:
        error_mssg = 'Updates were not pushed successfully to Structures. Please see Notebook output/logs for further details.'
        send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'structures push: FAILED!', e=error_mssg)
    else:
        success_mssg = 'There were ' + str(num_updates) + ' updates pushed to Production.'
        send_email('mycontacts.txt','success_structures_mssg.txt', subject = 'structures push: SUCCESS!', e=success_mssg)
else:
    mssg = 'There were no updates to push to the Structure layer.'
    send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'structures: NO UPDATES to push', e=mssg)

Dear Julie, 

This is a courtesy message to let you know updates to the Structure layer were pushed successfully.

Details:

There were 802 updates pushed to Production.

Thanks,
Data team at IPM



In [34]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

## Geosupport

### connect to geosupport service

In [35]:
geosupport_tbl_url = geosupport_url

In [36]:
tbl_geosupport = FeatureLayer(geosupport_tbl_url)
geosupport = tbl_geosupport.query()

### Get latest BINs from structures layer

In [37]:
structures_valid_BINs = structures_features.sdf[
    (structures_features.sdf['BIN'] != 0) &
    (~pd.isnull(structures_features.sdf['BIN']))] 

In [38]:
bins = structures_valid_BINs[structures_valid_BINs['BIN'].astype(int) %
                             100000 != 0]['BIN'].astype(int).to_list()

In [39]:
bins = [str(bins[i]) for i in range(0, len(bins))]

### run master geosupport function

In [40]:
df = master_geosupport_func(bins)

### prepare latest geosupport data to be pushed to prod

In [41]:
#Rename all of the columns from Geosupport so that they map exactly to the schema in SQL Server
dff = (df.rename(columns = {'out_zip_code': 'Zip_Code',
                               'b7sc': 'B7SC',
                               'out_b10sc1': 'B10SC',
                               'out_sanborn_boro': 'Boro_Code',
                               'bin': 'BIN',
                               'out_preferred_lgc': 'LGC',
                               'type_meaning': 'Address_Type',
                               'high_address_number': 'High_House_Num',
                               'low_address_number': 'Low_House_Num',
                               'out_sos_ind': 'Street_Side',
                               'street_name': 'Norm_Street_Name',
                               'out_usps_city_name': 'USPS_City',
                               'out_TPAD_bin_status': 'TPAD_BIN_Status',
                               'out_com_dist': 'Community_Board',
                               'out_co': 'City_Council',
                               'out_ad': 'NYS_Assembly',
                               'out_sd': 'NYS_Senate',
                               'out_cd': 'US_Congress',
                               'out_nta': 'NTA_Code',
                               'out_fire_bat': 'Fire_Battalion',
                               'out_fire_co': 'Fire_Co_Num',
                               'out_fire_co_str': 'Fire_Co_Type',
                               'out_fire_div': 'Fire_Division',
                               'out_hurricane_zone': 'HEZ',
                               'out_police_patrol_boro': 'Police_Boro',
                               'Police Patrol Borough Command': 'Police_Boro_Com',
                               'out_police_pct': 'Police_Precinct',
                               'Sanitation Collection Scheduling Section and Subsection': 'Sanitation_Subsect',
                               'Sanitation District': 'Sanitation_District',
                               'Sanitation Recycling Collection Schedule': 'Sanitation_Recycling',
                               'out_san_reg': 'Sanitation_Reg_Pickup',
                               'out_physical_id': 'Physical_ID',
                               'out_ap_id': 'Address_ID'})
       .reindex(columns = ['BIN', 'Boro_Code', 'Address_Type', 'Low_House_Num', 'High_House_Num', 'Norm_Street_Name', 'USPS_City', 
                           'Zip_Code', 'Physical_ID', 'B7SC', 'B10SC', 'LGC', 'Street_Side', 'TPAD_BIN_Status', 'HEZ', 
                           'Community_Board', 'City_Council', 'NYS_Assembly', 'NYS_Senate', 'US_Congress', 'NTA_Code', 
                           'Fire_Battalion', 'Fire_Co_Num', 'Fire_Co_Type', 'Fire_Division', 
                           'Police_Boro', 'Police_Boro_Com', 'Police_Precinct', 
                           'Sanitation_Subsect', 'Sanitation_District', 'Sanitation_Recycling', 'Sanitation_Reg_Pickup', 'Address_ID'])
       .drop_duplicates())

In [42]:
len(dff)

3543

In [43]:
dff.head(10)

,BIN,Boro_Code,Address_Type,Low_House_Num,High_House_Num,Norm_Street_Name,USPS_City,Zip_Code,Physical_ID,B7SC,...,Fire_Co_Type,Fire_Division,Police_Boro,Police_Boro_Com,Police_Precinct,Sanitation_Subsect,Sanitation_District,Sanitation_Recycling,Sanitation_Reg_Pickup,Address_ID
0,4592110,4,Non-Addressable Unnamed Building,None,None,BEACH 104 STREET,NaN,NaN,NaN,43361001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4453939,4,Ordinary Address Range,132A,132B,SERGEANT BEERS LANE,NaN,NaN,NaN,46189701,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4539845,4,Ordinary Address Range,340,340,STORY AVENUE,BAYSIDE,11359,0103875,46449501,...,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None,5164333
3,4539831,4,Ordinary Address Range,411A,411B,SHORE ROAD,NaN,NaN,NaN,46209501,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4539891,4,Ordinary Address Range,615,615,LITTLE BAY ROAD,BAYSIDE,11359,0103838,45410701,...,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None,5163935
5,4539854,4,Ordinary Address Range,620,620,CIRCLE DRIVE,BAYSIDE,11359,0165595,43902001,...,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None,5163980
6,4539910,4,Ordinary Address Range,417,417,WALTER REED ROAD,BAYSIDE,11359,0103869,46676001,...,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None,5164014
7,4539838,4,Ordinary Address Range,405,405,SYLVESTER LANE,BAYSIDE,11359,0103866,46504001,...,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None,5163945
8,4539848,4,Ordinary Address Range,319,319,LITTLE BAY ROAD,None,None,None,45410701,...,None,None,None,NaN,None,NaN,NaN,NaN,None,None
9,4539846,4,Ordinary Address Range,403,403,SYLVESTER LANE,BAYSIDE,11359,0103866,46504001,...,Engine 306,14,Queens North,NaN,109,NaN,NaN,NaN,None,5164208


In [44]:
measurer = np.vectorize(len)

In [ ]:
# type_df = pd.DataFrame({'df_name': dff.columns.tolist(),
#                         'df_type': dff.dtypes.astype(str).tolist(),
#                         'df_len' : measurer(dff.values.astype(str)).max(axis=0)})

In [ ]:
# type_df.head()

In [45]:
con_str = "Driver={SQL Server};Server=" + server + ";Database=structuresdb;Trusted_Connection=Yes;"
sa_con = quote_plus(con_str)
engine = sqlalchemy.create_engine(
    "mssql+pyodbc:///?odbc_connect={}".format(sa_con))

In [46]:
if len(dff) > 0:
    dff.to_sql('tbl_geosupport_address',
               engine,
               schema='dbo',
               if_exists='replace',
               index=False)

In [47]:
dff.Boro_Code.fillna(0,inplace=True)

In [48]:
dff.Address_ID.fillna(0,inplace=True)

In [49]:
geosupport_FSET = FeatureSet.from_dataframe(dff)

In [50]:
len(geosupport_FSET.features)

3543

### EDIT

In [55]:
if len(dff) > 0: #len(geosupport_FSET.features):
    try:
        ## TRUNCATE AND REPLACE
        tbl_geosupport.delete_features(where="objectid > 0")
        add_to_geosupport = tbl_geosupport.edit_features(adds = geosupport_FSET.features)
    except:
        error_mssg = 'Updates were not pushed successfully to the geosupport table. Please see Notebook output/logs for further details.'
        send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'geosupport push: FAILED!', e=error_mssg)
    else:
        success_mssg = 'There were ' + str(len(dff)) + ' updates pushed to Production.'
        send_email('mycontacts.txt','success_geosupport_mssg.txt', subject = 'geosupport push: SUCCESS!', e=success_mssg)
else:
    mssg = 'There were no updates to push to the geosupport table.'
    send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'geosupport: NO UPDATES to push', e=mssg)

Dear Julie, 

This is a courtesy message to let you know updates to the geosupport table were pushed successfully.

Details:

There were 3543 updates pushed to Production.

Thanks,
Data team at IPM

